In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip "/content/drive/MyDrive/IndoML/input_data.zip" -d "/content/drive/MyDrive/IndoML/Dataset/"

Archive:  /content/drive/MyDrive/IndoML/input_data.zip
  inflating: /content/drive/MyDrive/IndoML/Dataset/attribute_test.data  
  inflating: /content/drive/MyDrive/IndoML/Dataset/attribute_train.data  
  inflating: /content/drive/MyDrive/IndoML/Dataset/attribute_train.solution  
  inflating: /content/drive/MyDrive/IndoML/Dataset/attribute_val.data  
  inflating: /content/drive/MyDrive/IndoML/Dataset/attribute_val.solution  


In [1]:
!ls /content/drive/MyDrive/IndoML

Dataset  fast_text  input_data.zip  LG


In [3]:
!cd /content/drive/MyDrive/IndoML/last_day && git clone https://github.com/facebookresearch/fastText.git


Cloning into 'fastText'...
remote: Enumerating objects: 3998, done.
remote: Counting objects: 100% (1057/1057), done.
remote: Compressing objects: 100% (196/196), done.
remote: Total 3998 (delta 922), reused 884 (delta 856), pack-reused 2941 (from 1)
Receiving objects: 100% (3998/3998), 8.30 MiB | 7.31 MiB/s, done.
Resolving deltas: 100% (2529/2529), done.
Updating files: 100% (520/520), done.


In [4]:
!cd /content/drive/MyDrive/IndoML/last_day/fastText && mkdir build && cd build && cmake ..

CMake Deprecation Warning at CMakeLists.txt:9 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Configuring done (2.6s)
-- Generating done (0.6s)
-- Build files have been written to: /content/drive/MyDrive/IndoML/last_day/fastText/build


In [5]:
# prompt: make: *** No targets specified and no makefile found.  Stop.

!cd /content/drive/MyDrive/IndoML/last_day/fastText/build && make && make install


[ -2%] Building CXX object CMakeFiles/fasttext-shared.dir/src/args.cc.o
[  0%] Building CXX object CMakeFiles/fasttext-shared.dir/src/autotune.cc.o
[  2%] Building CXX object CMakeFiles/fasttext-shared.dir/src/densematrix.cc.o
[  4%] Building CXX object CMakeFiles/fasttext-shared.dir/src/dictionary.cc.o
[  6%] Building CXX object CMakeFiles/fasttext-shared.dir/src/fasttext.cc.o
[  8%] Building CXX object CMakeFiles/fasttext-shared.dir/src/loss.cc.o
[ 10%] Building CXX object CMakeFiles/fasttext-shared.dir/src/main.cc.o
[ 12%] Building CXX object CMakeFiles/fasttext-shared.dir/src/matrix.cc.o
[ 14%] Building CXX object CMakeFiles/fasttext-shared.dir/src/meter.cc.o
[ 17%] Building CXX object CMakeFiles/fasttext-shared.dir/src/model.cc.o
[ 19%] Building CXX object CMakeFiles/fasttext-shared.dir/src/productquantizer.cc.o
[ 21%] Building CXX object CMakeFiles/fasttext-shared.dir/src/quantmatrix.cc.o
[ 23%] Building CXX object CMakeFiles/fasttext-shared.dir/src/utils.cc.o
[ 25%] Building CXX

In [6]:
import string
import pandas as pd
from tqdm import tqdm

In [7]:
x_train_df = pd.read_json("/content/drive/MyDrive/IndoML/Dataset/attribute_train.data", lines=True)
y_train_df = pd.read_json("/content/drive/MyDrive/IndoML/Dataset/attribute_train.solution", lines=True)

print(x_train_df.shape, y_train_df.shape)

(443499, 4) (443499, 7)


In [8]:

x_val_df = pd.read_json("/content/drive/MyDrive/IndoML/Dataset/attribute_val.data", lines=True)
y_val_df = pd.read_json("/content/drive/MyDrive/IndoML/Dataset/attribute_val.solution", lines=True)

print(x_val_df.shape, y_val_df.shape)


(95035, 4) (95035, 7)


In [9]:
# Joing train and validation

x_train_df = pd.concat([x_train_df, x_val_df], axis=0)
y_train_df = pd.concat([y_train_df, y_val_df], axis=0)

print(x_train_df.shape, y_train_df.shape)

(538534, 4) (538534, 7)


In [10]:
x_test_df = pd.read_json("/content/drive/MyDrive/IndoML/Dataset/attribute_test.data", lines=True)
print(x_test_df.shape)

(95036, 4)


In [14]:
from nltk.corpus import stopwords
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_features(text):
  words = nltk.word_tokenize(text)
  words = [word for word in words if word not in stop_words]
  preprocessed_text = ' '.join(words)
  return preprocessed_text


def preprocess(df):
  df.fillna("None", inplace=True)
  df["features"] = "title: " + df["title"] + " " + "store: " + df["store"] + " " + "manufacturer: " + df["details_Manufacturer"]
  df["features"] = df["features"].str.lower()
  df['features'] = df['features'].str.replace('[{}]'.format(string.punctuation), '', regex=True)
  df['features'] = df['features'].str.replace('\d+', '')
  df['features'] = df['features'].apply(preprocess_features)

  df["details_Brand"] = df["details_Brand"].str.replace(' ', '%%')
  df["L0_category"] = df["L0_category"].str.replace(' ', '%%')
  df["L1_category"] = df["L1_category"].str.replace(' ', '%%')
  df["L2_category"] = df["L2_category"].str.replace(' ', '%%')
  df["L3_category"] = df["L3_category"].str.replace(' ', '%%')
  df["L4_category"] = df["L4_category"].str.replace(' ', '%%')
  df["label"] = '__label__' + df["details_Brand"] + '__' + df["L0_category"] + '__' + df["L1_category"] + '__' + df["L2_category"] + '__' + df["L3_category"] + '__' + df["L4_category"]
  df["training_row"] = df["label"] + ' ' + df["features"]
  return df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [15]:
x_train_df.sort_values(by="indoml_id", ascending=True, inplace=True)
y_train_df.sort_values(by="indoml_id", ascending=True, inplace=True)

train_df = pd.concat([x_train_df, y_train_df.drop("indoml_id", axis=1)], axis=1)

train_df = preprocess(train_df)

In [ ]:
# x_val_df.sort_values(by="indoml_id", ascending=True, inplace=True)
# y_val_df.sort_values(by="indoml_id", ascending=True, inplace=True)

# val_df = pd.concat([x_val_df, y_val_df.drop("indoml_id", axis=1)], axis=1)

# val_df = preprocess(val_df)

In [20]:
training_rows = train_df["training_row"].tolist()
with open("/content/drive/MyDrive/IndoML/last_day/indoml_try1.train", "w") as fp:
  for row in tqdm(training_rows):
    fp.write(row)
    fp.write("\n")

100%|██████████| 538534/538534 [00:01<00:00, 434397.49it/s]


In [ ]:
# validation_rows = val_df["training_row"].tolist()
# with open("/content/drive/MyDrive/IndoML/data/indoml.val", "w") as fp:
#   for row in tqdm(validation_rows):
#     fp.write(row)
#     fp.write("\n")

100%|██████████| 95035/95035 [00:00<00:00, 161547.63it/s]


In [ ]:
!cd /content/drive/MyDrive/IndoML/last_day/fastText && fasttext supervised -input /content/drive/MyDrive/IndoML/last_day/indoml_try1.train -epoch 7 -lr 1.0 -wordNgrams 2 -output /content/drive/MyDrive/IndoML/last_day/try1_model


Read 10M words
Number of words:  426140
Number of labels: 8362
Progress:   1.5% words/sec/thread:     734 lr:  0.985192 avg.loss: 16.139814 ETA:   2h19m36s

In [ ]:
!cd fastText && fasttext --help

usage: fasttext <command> <args>

The commands supported by fasttext are:

  supervised              train a supervised classifier
  quantize                quantize a model to reduce the memory usage
  test                    evaluate a supervised classifier
  test-label              print labels with precision and recall scores
  predict                 predict most likely labels
  predict-prob            predict most likely labels with probabilities
  skipgram                train a skipgram model
  cbow                    train a cbow model
  print-word-vectors      print word vectors given a trained model
  print-sentence-vectors  print sentence vectors given a trained model
  print-ngrams            print ngrams given a trained model and word
  nn                      query for nearest neighbors
  analogies               query for analogies
  dump                    dump arguments,dictionary,input/output vectors



In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.5-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.5-py3-none-any.whl (240 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4246563 sha256=9a9d2946c8133e230f6a8db21e780426736df3a27c6a1bf46104ee0479bab372
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
import fasttext

model = fasttext.load_model("/content/drive/MyDrive/IndoML/data/model_grams.bin")

x_test_df.fillna("None", inplace=True)
x_test_df["features"] = "title: " + x_test_df["title"] + " " + "store: " + x_test_df["store"] + " " + "manufacturer: " + x_test_df["details_Manufacturer"]

x_test_df["features"] = x_test_df["features"].str.lower()
x_test_df['features'] = x_test_df['features'].str.replace('[{}]'.format(string.punctuation), '', regex=True)
x_test_df['features'] = x_test_df['features'].str.replace('\d+', '')

test_rows = x_test_df["features"]

test_result_list = []
for index, row in tqdm(enumerate(test_rows)):
  prediction = model.predict(row)
  test_result_list.append(prediction[0][0])

ids = x_test_df["indoml_id"].tolist()

category_list = []
for index, row in enumerate(test_result_list):
  row = row.replace('__label__', '')
  details_Brand, L0_category, L1_category, L2_category, L3_category, L4_category = row.split('__')
  details_Brand = details_Brand.replace('%%', ' ')
  L0_category = L0_category.replace('%%', ' ')
  L1_category = L1_category.replace('%%', ' ')
  L2_category = L2_category.replace('%%', ' ')
  L3_category = L3_category.replace('%%', ' ')
  L4_category = L4_category.replace('%%', ' ')
  category_dict = {"indoml_id": ids[index], "details_Brand": details_Brand, "L0_category": L0_category, "L1_category": L1_category, "L2_category": L2_category, "L3_category": L3_category, "L4_category": L4_category}
  category_list.append(category_dict)

import json
with open("attribute_test_02092024.predict", "w") as fp:
  for row in category_list:
    fp.write(json.dumps(row))
    fp.write("\n")

95036it [02:29, 636.06it/s]


In [ ]:
!cd fastText && fasttext test model_3grams.bin /content/drive/MyDrive/IndoML/data/indoml.val

N	95035
P@1	0.929
R@1	0.929
